In [4]:
import requests
from bs4 import BeautifulSoup
import os
import zipfile

url = 'https://www.gov.br/ans/pt-br/acesso-a-informacao/participacao-da-sociedade/atualizacao-do-rol-de-procedimentos'

requisicao = requests.get(url)
pagina = BeautifulSoup(requisicao.text, 'html.parser')
links = pagina.find_all('a')
links_pdf = []

# Procura os Anexos I e II

for link in links:
    # Segundo parâmetro evita que ocorra um erro caso não exista um href
    href = link.get('href', '')
    if href.lower().endswith('.pdf'):
        if 'Anexo_I' in href or 'Anexo_II' in href:
            links_pdf.append(href)
            print(f'Link encontrado: {href}')

# Fazer download dos pdfs e compactar

pasta_downloads = 'downloads_ans'
if not os.path.exists(pasta_downloads):
    os.mkdir(pasta_downloads)

pdfs_baixados = []

for link_pdf in links_pdf:
    # Nome do arquivo (parametro 1: local a ser salvo, parametro 2: nome do arquivo)
    nome_arquivo = os.path.join(pasta_downloads, link_pdf.split('/')[-1])
    
    try:
        resposta_pdf = requests.get(link_pdf)
        
        with open(nome_arquivo, 'wb') as arquivo:
            arquivo.write(resposta_pdf.content)
        
        pdfs_baixados.append(nome_arquivo)
        print(f'Download concluído: {nome_arquivo}')
    
    except Exception as erro:
        print(f'Erro ao baixar {link_pdf}: {erro}')

# Se tiver algum valor dentro do Array Pdfs_baixados
if pdfs_baixados:
    pasta_atual = os.getcwd()
    caminho_zip = os.path.join(pasta_atual, 'anexos_ans.zip')
    
    with zipfile.ZipFile(caminho_zip, 'w') as arquivo_zip:
        for pdf in pdfs_baixados:
            arquivo_zip.write(pdf, os.path.basename(pdf))
    
    print(f'Arquivos compactados em {caminho_zip}')
else:
    print('Nenhum PDF encontrado')

Link encontrado: https://www.gov.br/ans/pt-br/acesso-a-informacao/participacao-da-sociedade/atualizacao-do-rol-de-procedimentos/Anexo_I_Rol_2021RN_465.2021_RN627L.2024.pdf
Link encontrado: https://www.gov.br/ans/pt-br/acesso-a-informacao/participacao-da-sociedade/atualizacao-do-rol-de-procedimentos/Anexo_II_DUT_2021_RN_465.2021_RN628.2025_RN629.2025.pdf
Download concluído: downloads_ans\Anexo_I_Rol_2021RN_465.2021_RN627L.2024.pdf
Download concluído: downloads_ans\Anexo_II_DUT_2021_RN_465.2021_RN628.2025_RN629.2025.pdf
Arquivos compactados em C:\Users\gabri\Workspace\teste-nivelamento\anexos_ans.zip
